### Linking Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


### Importing libraries

In [ ]:
import pandas as pd
import shutil
import os
import datetime 
import glob
import numpy as np
from openpyxl import load_workbook

#
import re

In [ ]:
len(os.listdir('/content/drive/MyDrive/08_Sales/01_Sales_Reports/D_Thrive/Frm April Summary report'))

192

### Copying Files

In [ ]:
source='/content/drive/MyDrive/Partners Reports 🧩/Thrive reports'
dest='/content/drive/MyDrive/01_Sales_Reports/D_Thrive/Frm April detailed report'
count=0

for file in os.listdir(source):
    if 'detailed_report' in file:
        count+=1
        src_path=os.path.join(source,file)
        des_path=os.path.join(dest,file)
        shutil.copy(src_path,des_path) 
print(count)

## Standard Format Consolidation

### Main DC code

In [ ]:
#@title Main Code
data=pd.DataFrame()
path = '/content/drive/MyDrive/Partners Reports 🧩/Thrive reports frm 230123-290123' # use your path

count=0

files = glob.glob(os.path.join(path , "*.csv")) ## getting each .csv files paths

for file in files:
    if 'summary' in file:
        read_csv=pd.read_csv(file,names=['Column1',	'Column2'	,	'Column3'	,'Column4', 'Column5','Column6'],skiprows=7)

        read_csv2=pd.read_csv(file,names=['Column1',	'Column2'	,	'Column3'	,'Column4', 'Column5'])

        read_csv.insert(1,'Period From',read_csv2.iloc[1]['Column2'],allow_duplicates=False)
        read_csv.insert(2,'Period To',read_csv2.iloc[2]['Column2'],allow_duplicates=False)
        read_csv.insert(3,'Res_name',read_csv2.iloc[0]['Column2'],allow_duplicates=False)
        
        data=pd.concat([data,read_csv])
        count+=1
        
print(count)

8


In [ ]:
data.head(2)

,Column1,Period From,Period To,Res_name,Column2,Column3,Column4,Column5,Column6
0,NaN,01-01-2023,31-01-2023,Mann Food Court,Delivery Orders,Take-Away Orders,Dine In Orders,Cancelled Orders,Total Orders
1,Number Of Orders,01-01-2023,31-01-2023,Mann Food Court,102,6,0,1,108


### Droping Null Rows

In [1]:
# Droping Null Rows
data.dropna(0,how='all',subset=['Column1',	'Column2'	,	'Column3'	,'Column4', 'Column5','Column6'],inplace=True);

### Droping rows and Renaming the cols

In [2]:
# Droping rows and Renaming the cols

data.dropna(0,how='any',subset=['Column1'],inplace=True)

r=	['Column1','Column2'	,	'Column3'	,'Column4', 'Column5','Column6']
d=['Particular','Delivery Orders',	'Take-Away Orders',	'Dine In Orders'	,'Cancelled Orders'	,'Total Orders']
rename_cols=dict(zip(r,d))
#rename_cols
data.rename(columns=rename_cols,inplace=True);

In [ ]:
data.head(3)

,Particular,Period From,Period To,Res_name,Delivery Orders,Take-Away Orders,Dine In Orders,Cancelled Orders,Total Orders
1,Number Of Orders,01-01-2023,31-01-2023,Mann Food Court,102,6,0,1,108
2,Item Sub-Total,01-01-2023,31-01-2023,Mann Food Court,42989.0,1611.0,0.0,288.0,44600.0
3,Packaging Or Service Charges,01-01-2023,31-01-2023,Mann Food Court,2685.0,115.0,0.0,25.0,2800.0


### Setting alphabetical serials for particulars

In [ ]:
#alpha serials for particulars
	
alpha_serials={'Number Of Orders':	'A',
              'Item Sub-Total':	'B',
              'Packaging Or Service Charges':	'C',
              'Delivery Charges'	:'D',
              'Discounts':	'E',
              'Net Bill Value'	:'F',
              'GST Charged to Customer':	'G',
              'Total Customer Payable':	'H',
              'Hashtag Loyalty Service Fees':	'I',
              'Razorpay Transaction Fee':	'J',
              'Delivery Fee':	'K',
              'Total GST over fees'	:'L',
              'Total Service Fees':	'M',
              'GST Deducted':	'N',
              'TCS Deducted'	:'O',
              'TDS Deducted':	'P',
              'Net Payable'	:'Q'}
data['Alpha_serials']=data['Particular'].map(alpha_serials)

### Adding Month and report periods columns & droping unneccessary colms

In [ ]:
#Adding Month and report periods, droping cols


data['Frm day']=data['Period From'].str.split('-',expand=True)[0]
data['Frm Mon']=data['Period From'].str.split('-',expand=True)[1]
data['Frm Mon name']=pd.to_datetime(data['Frm Mon'],format='%m').dt.month_name().str.slice(stop=3)


data['To day']=data['Period To'].str.split('-',expand=True)[0]
data['To Mon']=data['Period To'].str.split('-',expand=True)[1]
data['To Mon name']=pd.to_datetime(data['To Mon'],format='%m').dt.month_name().str.slice(stop=3)



data['New Period From']= data['Frm day'] + " " + data['Frm Mon name'].astype(str)
data['New Period To']= data['To day'] + " " + data['To Mon name'].astype(str)


data['Report Period']=data['New Period From']+"-"+data['New Period To']


data.drop(columns=['Frm Mon','Frm day','To day','To Mon','To Mon name','New Period From','New Period To'],inplace=True)
data.rename(columns={'Frm Mon name':'Month'},inplace=True)

In [ ]:
data.head(1)

,Particular,Period From,Period To,Res_name,Delivery Orders,Take-Away Orders,Dine In Orders,Cancelled Orders,Total Orders,Alpha_serials,Month,Report Period
1,Number Of Orders,01-01-2023,31-01-2023,Mann Food Court,102,6,0,1,108,A,Jan,01 Jan-31 Jan


In [ ]:
# Excluding Feb
data=data[~data['Month'].isin(['Feb'])]

### Setting Serial numbers for months 

In [ ]:
#@title Serial no. fr months
#data['Month'].unique()
months=['Mar','Apr','May','Jun', 'Jul','Aug','Sep','Oct','Nov', 'Dec','Jan']
serial_no=dict(zip(months,[mon for mon in range(1,len(months)+1)]))
#serial_no
data['Mon_Serialno']=data['Month'].map(serial_no)


#serial for particulars
l=list(data['Particular'].unique())

o=dict(zip(l,[i for i in range(1,len(l)+1)]))
                               
data['Serial_no']=data['Particular'].map(o)


### setting and Adding report_period_serials numbers colm

In [ ]:
#dding report_period_serials colm
report_period_serials={'28 Mar-03 Apr'	:1,
                        '04 Apr-10 Apr'	:2,
                        '11 Apr-17 Apr'	:3,
                        '18 Apr-24 Apr'	:4,
                        '25 Apr-01 May'	:5,
                        '30 May-05 Jun'	:10,
                        '23 May-29 May'	:9,
                        '02 May-08 May'	:6,
                        '16 May-22 May'	:8,
                        '09 May-15 May'	:7,
                        '20 Jun-26 Jun'	:13,
                        '13 Jun-19 Jun'	:12,
                        '27 Jun-03 Jul'	:14,
                        '06 Jun-12 Jun'	:11,
                        '25 Jul-31 Jul'	:18,
                        '18 Jul-24 Jul'	:17,
                        '11 Jul-17 Jul'	:16,
                        '04 Jul-10 Jul'	:15,
                        '29 Aug-04 Sep'	:23,
                        '01 Aug-07 Aug'	:19,
                        '22 Aug-28 Aug'	:22,
                        '15 Aug-21 Aug'	:21,
                        '08 Aug-14 Aug'	:20,
                        '19 Sep-25 Sep'	:26,
                        '26 Sep-02 Oct'	:27,
                        '05 Sep-11 Sep'	:24,
                        '12 Sep-18 Sep'	:25,
                        '17 Oct-23 Oct'	:30,
                        '10 Oct-16 Oct'	:29,
                        '24 Oct-30 Oct'	:31,
                        '03 Oct-09 Oct'	:28,
                        '31 Oct-06 Nov'	:32,
                        '28 Nov-04 Dec'	:35,
                        '14 Nov-20 Nov'	:34,
                        '07 Nov-13 Nov'	:33,
                        '26 Dec-01 Jan'	:39,
                        '12 Dec-18 Dec'	:37,
                        '05 Dec-11 Dec'	:36,
                        '19 Dec-25 Dec'	:38,
                        '09 Jan-15 Jan'	:41,
                        '02 Jan-08 Jan'	:40,
                        '16 Jan-22 Jan'	:42,
                        '23 Jan-29 Jan'	:43,}
data['Report_period_serial']=data['Report Period'].map(report_period_serials)

In [ ]:
data=data.sort_values(by='Report_period_serial')

In [ ]:
data.head(1)

,Particular,Period From,Period To,Res_name,Delivery Orders,Take-Away Orders,Dine In Orders,Cancelled Orders,Total Orders,Alpha_serials,Month,Report Period,Mon_Serialno,Serial_no,Report_period_serial
1,Number Of Orders,23-01-2023,29-01-2023,Mann Food Court,29,0,0,1,29,A,Jan,23 Jan-29 Jan,11,1,43.0


### Searching for Whole month periods in report period column and excluding them - 1

In [ ]:
# Making Not Needed periods
u=list(data['Report Period'].unique())
whole_month_periods=[]
j=['Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov','Dec', 'Jan']
for i in u:
    for k in j:
        if f'01 {k}-31 {k}' in i:
            whole_month_periods.append(i)
        if f'01 {k}-30 {k}' in i:
            whole_month_periods.append(i) 
      #print(i)
# #whole_month_periods=['01 Mar-31 Mar','01 Jul-31 Jul','01 Apr-30 Apr','01 Jul-31 Jul','01 Aug-31 Aug','01 Sep-30 Sep','01 Oct-31 Oct','01 Nov-30 Nov','01 Dec-31 Dec']


### Searching for Whole month periods in report period column and excluding them - 2

In [ ]:
# Seprating months data and savng to excel
month_data=data[data['Report Period'].isin(whole_month_periods)]
month_data=month_data.sort_values(by='Mon_Serialno')
month_data.drop(columns=['Report_period_serial'],inplace=True)
month_data.rename(columns={'Report Period':'Month Report Period'},inplace=True)
month_data.head(3)


,Particular,Period From,Period To,Res_name,Delivery Orders,Take-Away Orders,Dine In Orders,Cancelled Orders,Total Orders,Alpha_serials,Month,Month Report Period,Mon_Serialno,Serial_no
1,Number Of Orders,01-01-2023,31-01-2023,Mann Food Court,102,6,0,1,108,A,Jan,01 Jan-31 Jan,11,1
3,Packaging Or Service Charges,01-01-2023,31-01-2023,Chinese Chaska,450.0,80.0,0.0,50.0,530.0,C,Jan,01 Jan-31 Jan,11,3
4,Delivery Charges,01-01-2023,31-01-2023,Chinese Chaska,795.0,0.0,0.0,55.0,795.0,D,Jan,01 Jan-31 Jan,11,4


### Excluding Not Needed periods

In [ ]:
# Excluding Not Needed periods
data=data[~data['Report Period'].isin(whole_month_periods)]

### Saving as excel in drive

In [ ]:
data.to_excel('/content/drive/MyDrive/13_Data_Analyst/Thrive/Thrive Dashboard File/Thrive frm April 2022.xlsx')

## Pivoting data📌

In [ ]:
#@title Pivoted data
pivoted_data=pd.pivot_table(data,values='Total Orders',index=['Serial_no','Particular','Res_name'],columns=['Mon_Serialno','Report Period'],aggfunc='sum').sort_values(by='Serial_no')
#pivoted_data.head(3)


### Creating New DataFrame

In [ ]:
#Creating New DataFrame
db=pd.DataFrame(pivoted_data)
db=db.reset_index()
db.fillna(0,inplace=True)
db.columns=db.columns.droplevel(0)

db.columns.values[0]='Serial_no'
db.columns.values[2]='Res_name'
db.columns.values[1]='Particular'
db.head(2)

Report Period,Serial_no,Particular,Res_name,23 Jan-29 Jan
0,1,Number Of Orders,Chinese Chaska,9
1,1,Number Of Orders,Dilli Darbar,20


### import google sheets libraries

In [ ]:
# import google sheets libraries
from google.colab import auth
from google.colab import auth
from google.auth import default
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
import gspread

auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

### opening different sheets and also appedning from last row of data in sheet

In [ ]:
# opening different sheets and also appedning from last row
import gspread_dataframe as gd
wc=gc.open_by_key('1ZYlLQJ2Nq_Isk6jsaGFaVfJ714U5TgF8k6mELPiZIY0')

#wd=gc.open_by_url('https://docs.google.com/spreadsheets/d/1ZYlLQJ2Nq_Isk6jsaGFaVfJ714U5TgF8k6mELPiZIY0/edit#gid=643907605')

new=wc.worksheet('Month wise')
rows=new.get_all_values()
df12=pd.DataFrame.from_records(rows[1:],columns=rows[0]) # viewing the google sheet data
#df12

existing_data=gd.get_as_dataframe(new)
update=existing_data.append(month_data)
gd.set_with_dataframe(new,update)



In [ ]:
# update the google sheet by by cell 


cell_list = worksheet.range('AT2:AT500')
cell_values = db['16 Jan-22 Jan'].tolist()

for i, val in enumerate(cell_values):  #gives us a tuple of an index and value
    cell_list[i].value = val    #use the index on cell_list and the val from cell_values

worksheet.update_cells(cell_list)
worksheet.update_acell('AT1','16 Jan-22 Jan')

### Adding Credit% Rows to pivoted data and saving

In [ ]:
# Adding Credit% Rows
p=db[db['Particular']=='Net Bill Value'].groupby(['Res_name'])[db.columns.values[3:]].sum()
#p



e=db[db['Particular']=='Net Payable'].groupby(['Res_name'])[db.columns.values[3:]].sum()
#e

Credit_percent=pd.DataFrame(round(((p/e)*100),0))
Credit_percent.insert(0,'Particular',value='Credit%',allow_duplicates=False)
Credit_percent=Credit_percent.reset_index() 
Credit_percent


# credit_per_order= 'net payable'/No. of orders


db.to_excel('/content/drive/MyDrive/13_Data_Analyst/Thrive/Thrive Dashboard File/Thrive frm April 2022 pivoted.xlsx')